## Network creation

Using the transformed data from the previous step, create networks for February and April.

In [1]:
# to allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)

import networkx as nx
import pickle

from data_processing.data_paths import OUT, GRAPHS
from model.network.network_data import NetworkData
from model.network.mobility_network import MobilityNetwork
from model.network.mobility_network import MNGeneratorFromNetworkData as MNG

Define a seed for random generators.

In [2]:
SEED = 4401

Read transformed data from pickled files.

In [3]:
demographics = pickle.load(open(f'{OUT}demographics.pkl', 'rb'))

comb_counts_pre = pickle.load(open(f'{OUT}comb_counts_pre.pkl', 'rb'))
comb_counts_post = pickle.load(open(f'{OUT}comb_counts_post.pkl', 'rb'))

trip_counts_pre = pickle.load(open(f'{OUT}trip_counts_pre.pkl', 'rb'))
trip_counts_post = pickle.load(open(f'{OUT}trip_counts_post.pkl', 'rb'))

Create network data instances.

In [4]:
network_data_pre = NetworkData(demographics, comb_counts_pre, trip_counts_pre)
network_data_post = NetworkData(demographics, comb_counts_post, trip_counts_post)

network_data_post.calc_trip_count_change(network_data_pre)

Calculate the adjacency lists and cumulative probabilities.

In [5]:
network_data_pre.create_adjacency_list()
network_data_post.create_adjacency_list()

network_data_pre.create_cum_prob()
network_data_post.create_cum_prob()

Create the networks.

In [6]:
%%time

params = dict()
params[MNG.N] = N = 10000
params[MNG.EXPONENT] = EXPONENT = 2
params[MNG.CUTOFF] = CUTOFF = 10
params[MNG.SEED] = SEED

params_pre = params_post = params

params_pre[MNG.NETWORK_DATA] = network_data_pre
params_pre[MNG.MULTIPLIER] = False

params_post[MNG.NETWORK_DATA] = network_data_post
params_post[MNG.MULTIPLIER] = True

g_pre = MNG(params=params_pre).generate()
g_post = MNG(params=params_post).generate()

CPU times: user 1min 44s, sys: 84.4 ms, total: 1min 44s
Wall time: 1min 44s


Save the graphs to graphml format.

In [7]:
nx.write_graphml(g_pre, f'{GRAPHS}pre_N_{N}_E_{EXPONENT}_C_{CUTOFF}.graphml')
nx.write_graphml(g_post, f'{GRAPHS}post_N_{N}_E_{EXPONENT}_C_{CUTOFF}.graphml')